In [ ]:
pip install qiskit qiskit_aer pylatexenc

User's choice: define 2 booleans $(x_1, x_2) \in \mathbb{F}_2$ and an error probability $p \in [0, 1]$ for the random Pauli error.

In [ ]:
x1, x2 = 0, 0
p = 0.1

Step 1: Create register for the $5$ qubits

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister

qr = QuantumRegister(5, "q")
qcirc = QuantumCircuit(qr)

Step 2: Prepare the logical $|x_L>$ state based on the given booleans x1, x2. Save the decoded, initialized qubits to retrieve them later.

In [ ]:
import numpy as np

if (x1,x2)==(0,0):
    alpha, beta = 1,0
elif (x1,x2)==(0,1):
    alpha, beta = 0,1
elif (x1,x2)==(1,0):
    alpha, beta = 1/np.sqrt(2),  1/np.sqrt(2)
elif (x1,x2)==(1,1):
    alpha, beta = 1/np.sqrt(2), -1/np.sqrt(2)

θ = 2*np.arccos(np.abs(alpha))
φ = np.angle(beta) - np.angle(alpha)
if abs(θ)>1e-12:
    qc.ry(θ, qr[0])
if abs(φ)>1e-12:
    qc.rz(φ, qr[0])

qcirc.h(qr[0])
qcirc.cx(qr[0], qr[2])
qcirc.cx(qr[2], qr[1])
qcirc.cx(qr[1], qr[3])
qcirc.cx(qr[3], qr[4])
qcirc.cz(qr[0], qr[4])

dcirc = qcirc.inverse() # decoder for later use

# visualize
qcirc.barrier()
qcirc.draw(output='mpl')

Step 3: Add one random Pauli error on each of the $5$ qubits with probability $p$.

In [ ]:
for i in range(5):
    r = np.random.rand()
    if r < p/3:
        qcirc.x(qr[i])
    elif r < 2*p/3:
        qcirc.y(qr[i])
    elif r < p:
        qcirc.z(qr[i])

# visualize
qcirc.barrier()
qcirc.draw(output='mpl')

Step 4: Use stabilizer components to detect errors as syndrome in $4$ ancillas.

In [ ]:
ar    = QuantumRegister(4, "a")
qcirc.add_register(ar)

P = ['IXZZX',
     'XIXZZ',
     'ZXIXZ',
     'XZZXI']

for i in range(len(P)):
    qcirc.h(ar[i])
    for j in range(len(P[i])):
        if P[i][j] == 'X':
            qcirc.cx(qr[j], ar[i])
        if P[i][j] == 'Z':
            qcirc.cz(qr[j], ar[i])
    qcirc.h(ar[i])

 # visualize
qcirc.barrier()
qcirc.draw(output='mpl')

Step 5: Mesure the syndrome as classical bits.

In [ ]:
from qiskit import ClassicalRegister

sy   = ClassicalRegister(4, "s")
qcirc.add_register(sy)

qcirc.measure(ar[0], sy[0])
qcirc.measure(ar[1], sy[1])
qcirc.measure(ar[2], sy[2])
qcirc.measure(ar[3], sy[3])

 # visualize
qcirc.barrier()
qcirc.draw(output='mpl')

Step 6: Correct the error on the qubits based on the detected syndrome.

In [ ]:
corrections = {
# no error
'0000': ('I', None),

# qubit 1
'1110': ('X', 0),
'0111': ('Y', 0),
'1001': ('Z', 0),

# qubit 2
'1011': ('X', 1),
'1101': ('Y', 1),
'0100': ('Z', 1),

# qubit 3
'1100': ('X', 2),
'1010': ('Y', 2),
'0011': ('Z', 2),

# qubit 4
'1000': ('X', 3),
'1111': ('Y', 3),
'0110': ('Z', 3),

# qubit 5
'0101': ('X', 4),
'0001': ('Y', 4),
'1110': ('Z', 4)
}

for s_type, (corr, qidx) in corrections.items():
    if corr == 'I':
        continue
    s_int = int(s_type[::-1], 2)
    with qcirc.if_test((sy, s_int)):
        if corr == 'X':
            qcirc.x(qr[qidx])
        elif corr == 'Y':
            qcirc.y(qr[qidx])
        elif corr == 'Z':
            qcirc.z(qr[qidx])

 # visualize
qcirc.barrier()
qcirc.draw(output='mpl')

Step 7: Use the decoder to measure the first single logical qubit.

In [ ]:
qcirc.compose(dcirc, qr, inplace=True)
mr    = ClassicalRegister(1, "m")   # final logical measurement after decode
qcirc.add_register(mr)
qcirc.measure(qr[0], mr[0])

 # visualize
qcirc.barrier()
qcirc.draw(output='mpl')

Step 8: Based on the user's entry for the boolean values, check if the simulaed quantum circuit returns what's expected.

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import transpile

simulator = AerSimulator()
compiled_circuit = transpile(qcirc, simulator)
job = simulator.run(compiled_circuit, shots=500)
counts = job.result().get_counts()

bit_str = max(counts, key=counts.get)
bit = int(bit_str.replace(" ", ""), 2)

if (x1, x2) == (0, 0) or (x1, x2) == (1, 0):
  if bit == 0:
    print('Correct')
  else:
    print('Failed')
else:
  if bit == 1:
    print('Correct')
  else:
    print('Failed')

